In [1]:
import pandas as pd 
import requests
import os
import json
import numpy as np

In [2]:
#import data
data = pd.read_csv(r'C:\Users\akobe\OneDrive\Desktop\Lighthouse\After\NHL-Stats\Player_Info_23_24\Data\roster_data.csv', index_col = [0]) #index col makes sures another index column is not needed

In [5]:
# column to list using Series.tolist()
player_code_list = data['player_id'].tolist()

In [8]:
#getting url's for player info
all_url = []
for i in range(len(player_code_list)):
    player_code = player_code_list[i]
    player_url = 'https://api-web.nhle.com/v1/player/'+str(player_code)+'/landing'
    all_url.append(player_url)

In [10]:
player_data = []

for i in range(len(all_url)):
    response = requests.get(all_url[i])
    player_data.append(response.json())

In [13]:
#save a JSON file
#with open("player_data.json", "w") as file:
#    json.dump(player_data, file)

## Creating draft data dataframe 

In [2]:
#import the json file 
f = open(r'C:\Users\akobe\OneDrive\Desktop\Lighthouse\After\NHL-Stats\Player_Info_23_24\Data\player_data.json')
player_data = json.load(f)

In [11]:
#from IPython.display import JSON
#JSON(player_data)

In [18]:
#empty lists to create the draft data frame 
player_id = []
current_team = []
first_name = []
last_name = []
draft_year = []
draft_team = []
draft_round = []
pick_in_round = []
pick_overall = []

In [19]:
#pull data from JSON into a list - for draft dataframe 
for i in range(len(player_data)):
    player_id.append(player_data[i]['playerId'])
    current_team.append(player_data[i]['currentTeamAbbrev'])
    first_name.append(player_data[i]['firstName']['default'])
    last_name.append(player_data[i]['lastName']['default'])
    try:
        draft_year.append(player_data[i]['draftDetails']['year'])
        draft_team.append(player_data[i]['draftDetails']['teamAbbrev'])
        draft_round.append(player_data[i]['draftDetails']['round'])
        pick_in_round.append(player_data[i]['draftDetails']['pickInRound'])   
        pick_overall.append(player_data[i]['draftDetails']['overallPick'])
    except KeyError: 
        draft_year.append('NaN')
        draft_team.append('NaN')
        draft_round.append('NaN')
        pick_in_round.append('NaN')   
        pick_overall.append('NaN')

In [20]:
#create a data frame with the draft info
draft_data = pd.DataFrame(list(zip(player_id, current_team, first_name, last_name, draft_year, draft_team, 
                                   draft_round, pick_in_round, pick_overall)),
               columns =['player_id', 'current_team', 'first_name', 'last_name', 'draft_year', 'draft_team', 
                                   'draft_round', 'pick_in_round', 'pick_overall'])

In [24]:
draft_data.head()

,player_id,current_team,first_name,last_name,draft_year,draft_team,draft_round,pick_in_round,pick_overall
0,8484153,ANA,Leo,Carlsson,2023,ANA,1,2,2
1,8482118,ANA,Sam,Colangelo,2020,ANA,2,5,36
2,8483445,ANA,Cutter,Gauthier,2022,PHI,1,5,5
3,8480870,ANA,Bo,Groulx,2018,ANA,2,23,54
4,8477527,ANA,Ross,Johnston,NaN,NaN,NaN,NaN,NaN


In [22]:
#create a csv of the draft info
draft_data.to_csv('draft_data.csv')